In [1]:
import pandas as pd
import os

In [2]:
DATA_DIR = 'integration_data'

gene_mapping_filepath = os.path.join(DATA_DIR, 'gene_name_map.txt') # text file generated from HGCN website
cui_to_hgnc_filepath = os.path.join(DATA_DIR, 'HGNC_to_CUI.csv') # CUI to HGNC mapping from UMLS 
gene_mapping_output_filepath = os.path.join(DATA_DIR, 'gene_mapping.csv') # where the formatted gene mapping is to be ouputted

In [3]:
gene_name_map_df = pd.read_csv(gene_mapping_filepath, sep="\t", dtype=str)

In [4]:
# add CUI
cui_to_hgnc = pd.read_csv(cui_to_hgnc_filepath)
gene_name_map_df = gene_name_map_df.merge(cui_to_hgnc, how='left', on='HGNC ID').rename(columns={"CUI": "CUI_gene"})

In [5]:
# save to csv
gene_name_map_df.to_csv(gene_mapping_output_filepath, index=False)

In [6]:
gene_name_map_df.head()

,HGNC ID,Approved symbol,Alias symbols,UniProt ID(supplied by UniProt),OMIM ID(supplied by OMIM),NCBI Gene ID(supplied by NCBI),Ensembl ID(supplied by Ensembl),CUI_gene
0,HGNC:1,A12M1,NaN,NaN,NaN,NaN,NaN,NaN
1,HGNC:10,A2MRAP,NaN,NaN,NaN,NaN,NaN,NaN
2,HGNC:100,ASIC1,"BNaC2, hBNaC2",P78348,602866,41,ENSG00000110881,C1412115
3,HGNC:1000,BCL5,NaN,NaN,NaN,NaN,NaN,C1412765
4,HGNC:10000,RGS4,NaN,P49798,602516,5999,ENSG00000117152,C1419374
